# Goal
Jacobo de la Cuesta-Zuluaga.

According to the `sourmash` result, after metagenome assembly and binning, there were two bins classified as *Archaea*. One of them is classified as *Methanobrevibacter smithii*, while the second one is only classified up to kingdom. We suspect this bin to be **vadinC11**. Here, I will assess the quality of the potential **vadicC11** bin assembled using shotgun reads from the 81 and 93 sequencing runs of the TwinsUK samples.

# Var

In [1]:
# Work dir
work_dir = "/ebio/abt3_projects/vadinCA11/data/V11"

# vadinCA11 genome bin
V11_bin = "/ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/metabat2_low_PE.478.contigs.fa"
V11_folder = "/ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/"

# Misc
quality_env = "py2_genome_quality"
anvio_env = "anvio"
metacompass_env = "metacompass"

# anvio coassemble
contig_db_file = '/ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/anvio/coassemble.db'

# twinsUK
tax_file = '/ebio/abt3_projects/TwinsUK/Dataset_summary/TwinsUK_Batch3/TwinsUK_Batch3_taxonomy.txt'
seqs_file = '/ebio/abt3_projects/TwinsUK/Dataset_summary/TwinsUK_Batch3/rep_set.fna'

# SILVA 132 database
silva_align = "/ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/97/silva_132_97_16S.fna"
silva_tax = "/ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"

# Init

In [2]:
import os
import pandas as pd
import subprocess
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML

# Confirm 16S rRNA gene
The Main objective of this section is to confirm that the bin classified as *unclassified archaeon* indeed corresponds to **vadinCA11** using the 16S rRNA gene

## Getting archaeal 16S genes from anvio db
* Aim: To extract all 16S rRNA archaeal reads from the coassembly

In [3]:
# listing available genes
list_cmd = """anvi-get-sequences-for-hmm-hits \
  -c {0} \
  --hmm-source Ribosomal_RNAs \
   --list-available-gene-names"""
list_cmd = list_cmd.format(contig_db_file)
list_job = 'bash -c "source activate {0}; {1}"'
list_job = list_job.format(anvio_env, list_cmd)
print(list_job)
!$list_job

bash -c "source activate anvio; anvi-get-sequences-for-hmm-hits   -c /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/anvio/coassemble.db   --hmm-source Ribosomal_RNAs    --list-available-gene-names"
* Ribosomal_RNAs [type: Ribosomal_RNAs]: Archaeal_16S_rRNA, Archaeal_23S_rRNA,
Archaeal_5S_rRNA, Bacterial_16S_rRNA, Bacterial_23S_rRNA, Bacterial_5S_rRNA,
Eukaryotic_28S_rRNA, Eukaryotic_5S_rRNA, Eukaryotic_5_8S_rRNA,
Mitochondrial_12S_rRNA, Mitochondrial_16S_rRNA, Archaeal_5_8S_rRNA




In [4]:
# writing out genes
archaeal_16S_file = os.path.join(work_dir, "16S", 'archaeal_16SrRNA.fna')
w16S_cmd = """anvi-get-sequences-for-hmm-hits \
  -c {0} \
  --hmm-source Ribosomal_RNAs \
  --gene Archaeal_16S_rRNA \
  -o {1}"""
w16S_cmd = w16S_cmd.format(contig_db_file, archaeal_16S_file)
w16S_job = 'bash -c "source activate {}; {}"'
w16S_job = w16S_job.format(anvio_env, w16S_cmd)
print(w16S_job)
!$w16S_job

bash -c "source activate anvio; anvi-get-sequences-for-hmm-hits   -c /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/anvio/coassemble.db   --hmm-source Ribosomal_RNAs   --gene Archaeal_16S_rRNA   -o /ebio/abt3_projects/vadinCA11/data/V11/16S/archaeal_16SrRNA.fna"
Contigs DB ...................................: Initialized: /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/anvio/coassemble.db (v. 10)
Hits .........................................: 75 hits for 1 source(s)
Filtered hits ................................: 0 hits remain after filtering for 1 gene(s)


Config Error: Your selections resulted in 0 hits. There is nothing to report. Are you sure you
              have the right set of gene names and sources? If you select an HMM source, and  
              then use gene names that belong to another source, the intersection of the two  
              can be empty. Just saying.                                                      




In [5]:
!grep ">" $archaeal_16S_file

grep: /ebio/abt3_projects/vadinCA11/data/V11/16S/archaeal_16SrRNA.fna: No such file or directory


For some reason the extraction of 16S rRNA sequences from the coassembly using anvio doesn't work. I will verify what is happening latter, but for now I will take a different approach. Instead of extracting the 16S sequences from the complete coassembly, I will do it from the **vadinCA11** bin, and will work from there.

## Getting vadinCA11 16S genes
* Aim: To extract the 16S amplicon reads classified as **vadinCA11** from the TwinsUK data

In [6]:
# querying tax file
vadinCA11_tax_file = os.path.join(work_dir, "16S", 'vadinCA11_tax.txt')
!grep -i "vadinCA11" $tax_file > $vadinCA11_tax_file
!wc -l $vadinCA11_tax_file

12 /ebio/abt3_projects/vadinCA11/data/V11/16S/vadinCA11_tax.txt


In [7]:
# querying tax file
tmp_file = os.path.join(work_dir, "16S", 'tmp.txt')
!cut -f 1 $vadinCA11_tax_file | perl -pe 's/^/>/;s/$/ /'> $tmp_file
!wc -l $tmp_file
!cat $tmp_file

12 /ebio/abt3_projects/vadinCA11/data/V11/16S/tmp.txt
>565570 
>4442338 
>New.0.CleanUp.ReferenceOTU19387 
>New.0.CleanUp.ReferenceOTU25898 
>152715 
>4011307 
>New.3.CleanUp.ReferenceOTU8270 
>New.10.CleanUp.ReferenceOTU2657 
>New.11.CleanUp.ReferenceOTU470 
>New.17.CleanUp.ReferenceOTU26822 
>New.26.CleanUp.ReferenceOTU28048 
>New.32.CleanUp.ReferenceOTU2934 


In [8]:
vadinCA11_seqs_file = os.path.join(work_dir, "16S", 'vadinCA11_rep.fna')
!egrep -A 1 -f $tmp_file $seqs_file | perl -ne 'print unless /^--$/' > $vadinCA11_seqs_file
!grep -c ">" $vadinCA11_seqs_file
!head $vadinCA11_seqs_file

12
>4442338 High1.23_142512
TACCCGCAGCCCAAGTGGTGGTCGATTTTATTGAGTCTAAAACGTTCGTAGCCGGTTCATTAAATCCTTGGGTAAATCGGGAAGCTTAACTTTCCGACTTCCGAGGAGACTGGTGAACTTGGGACCGGGAGAGGCAAGAGGTACTTCTGGGGTAGGGGTAAAATCCTGTAATCCTAGAAGGACCACCGGTGGCGAAGGCGTCTTGCTAGAACGGATCCGACGGTGAGGGACGAAGCCCTGGGTCGCAAACGGG
>565570 High1.67_26391
TACCTGCAGCCCAAGTGGTGGTCGATTTTATTGAGTCTAAAACGTTCGTAGCCGGTCTGGTAAATCCTTGGGTAAATCGGGAAGCTTAACTTTCCGACTTCCGAGGAGACTGCCAGACTTGGGACCGGGAGAGGCAAGAGGTACTTCTGGGGTAGGGGTAAAATCCTGTAATCCTAGAAGGACCACCGGTGGCGAAGGCGTCTTGCTAGAACGGATCCGACGGTGAGGGACGAAGCCCTGGGTCGCAAACGGG
>New.0.CleanUp.ReferenceOTU19387 High1.82_4039280
TACCTGCAGCCCAAGTGGTGGTCGATTTTATTGAGTCTAAAACGTTCGTAGCCGGTCTGGTAAATCCTTGGGTAAATCGGGGAGCTCAACTTTCCGAATTCCGAGGAGACTGCCAGACTTGGGACCGGGAGAGGCTAGAGGTACTTCTGGGGTAGGGGTAAAATCCTGTAATCCTAGAAGGACCACCGGTGGCGAAGGCGTCTAGTTAGAACGGATCCGACGGTGAGGGACGAAGCCCTGGGTCGCAAACGGG
>New.0.CleanUp.ReferenceOTU25898 High1.23_1032914
TACCCGCAGCCCGAGTGGTGGTCGATTTTATTGAGTCTAAAACGTTCGTAGCCGGTCTGATAGATCCTTGGGTAAATCGGGGGGCTT

## Blasting 16S amplicons to the vadinCA11 bin
* Aim: to blast the 16S amplicons classified as **VadinCA11** to the candidate bin to determine their similarity. 

In [9]:
# Run blast job
blast_res_file = os.path.join(work_dir, "16S", 'V11_bin_16S.txt')
blast_cmd = 'blastn -query {query} -subject {subject} -outfmt 6 > {output}'
blast_cmd = blast_cmd.format(query = vadinCA11_seqs_file,
                 subject = V11_bin,
                 output = blast_res_file)
blast_job = 'bash -c "source activate {}; {}"'
blast_job = blast_job.format(anvio_env, blast_cmd)
print(blast_job)
!$blast_job

bash -c "source activate anvio; blastn -query /ebio/abt3_projects/vadinCA11/data/V11/16S/vadinCA11_rep.fna -subject /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/metabat2_low_PE.478.contigs.fa -outfmt 6 > /ebio/abt3_projects/vadinCA11/data/V11/16S/V11_bin_16S.txt"


In [10]:
# Print table with blast results
bin16S_df = pd.read_csv(blast_res_file, sep='\t', header=None)
bin16S_df.columns = ['qseqid','sseqid','pident','length','mismatch',
              'gapopen','qstart','qend','sstart','send','evalue','bitscore']
bin16S_df

,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore
0,4442338,coassemble_31453,91.37,255,18,4,1,253,414,162,4.000000e-98,346
1,565570,coassemble_31453,92.49,253,19,0,1,253,414,162,4.000000e-103,363
2,New.0.CleanUp.ReferenceOTU19387,coassemble_31453,91.70,253,21,0,1,253,414,162,8.000000e-100,351
3,New.0.CleanUp.ReferenceOTU25898,coassemble_31453,100.00,253,0,0,1,253,414,162,8.000000e-135,468
4,152715,coassemble_31453,90.51,253,24,0,1,253,414,162,8.000000e-95,335
5,4011307,coassemble_31453,91.30,253,22,0,1,253,414,162,4.000000e-98,346
6,New.3.CleanUp.ReferenceOTU8270,coassemble_31453,91.70,253,21,0,1,253,414,162,8.000000e-100,351
7,New.10.CleanUp.ReferenceOTU2657,coassemble_31453,88.41,207,24,0,1,207,414,208,3.000000e-69,250
8,New.11.CleanUp.ReferenceOTU470,coassemble_31453,92.27,207,16,0,1,207,414,208,1.000000e-82,294
9,New.17.CleanUp.ReferenceOTU26822,coassemble_31453,90.91,253,23,0,1,253,414,162,2.000000e-96,340


## Extracting 16S genes from V11 bin
* Aim: To extract the 16S rRNA gene present in the **vadinCA11** bin and to determine the closest matches in SILVA.

For this, I will extract it using `barrnap` and use blast to compare it to sequences in the SILVA 132 database (clustered at 97% identity).

In [11]:
# Extract 16S sequences from the assembled contigs using barrnap
barrnap_fasta = os.path.join(work_dir, "16S", 'barrnap_rRNA.fa')
barrnap_gff = os.path.join(work_dir, "16S", 'barrnap_rRNA.gff')
barrnap_cmd = "barrnap --kingdom {0} --threads {1} --outseq {2} < {3} > {4}"
barrnap_cmd = barrnap_cmd.format("arc", 8, barrnap_fasta, V11_bin, barrnap_gff)
barrnap_job = 'bash -c "source activate {}; {}"'
barrnap_job = barrnap_job.format(metacompass_env, barrnap_cmd)
print(barrnap_job)
!$barrnap_job

bash -c "source activate metacompass; barrnap --kingdom arc --threads 8 --outseq /ebio/abt3_projects/vadinCA11/data/V11/16S/barrnap_rRNA.fa < /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/metabat2_low_PE.478.contigs.fa > /ebio/abt3_projects/vadinCA11/data/V11/16S/barrnap_rRNA.gff"
[barrnap] This is barrnap 0.9
[barrnap] Written by Torsten Seemann
[barrnap] Obtained from https://github.com/tseemann/barrnap
[barrnap] Detected operating system: linux
[barrnap] Adding /ebio/abt3_projects/software/miniconda3/envs/metacompass/lib/barrnap/bin/../binaries/linux to end of PATH
[barrnap] Checking for dependencies:
[barrnap] Found nhmmer - /ebio/abt3_projects/software/miniconda3/envs/metacompass/bin/nhmmer
[barrnap] Found bedtools - /ebio/abt3_projects/software/miniconda3/envs/metacompass/bin/bedtools
[barrnap] Will use 8 threads
[barrnap] Setting evalue cutoff to 1e-06
[barrnap] Will tag genes < 0.8 of expected length.
[barrnap] Will rej

In [12]:
# Print all rRNA genes found by `barrnap`
V11_rRNA_df = pd.read_csv(barrnap_gff, sep='\t', skiprows=1, header=None)
V11_rRNA_df.columns = ['seqname','source','feature','start','end', 'score', 
                       'strand','frame','attribute']
V11_rRNA_df

,seqname,source,feature,start,end,score,strand,frame,attribute
0,coassemble_31453,barrnap:0.9,rRNA,1,878,7.600000e-212,-,.,Name=16S_rRNA;product=16S ribosomal RNA (parti...
1,coassemble_55654,barrnap:0.9,rRNA,19,124,4.400000e-11,-,.,Name=5S_rRNA;product=5S ribosomal RNA
2,coassemble_59052,barrnap:0.9,rRNA,19,124,4.400000e-11,-,.,Name=5S_rRNA;product=5S ribosomal RNA
3,coassemble_60057,barrnap:0.9,rRNA,8234,8339,4.400000e-11,-,.,Name=5S_rRNA;product=5S ribosomal RNA
4,coassemble_80719,barrnap:0.9,rRNA,2,680,6.300000e-177,+,.,Name=16S_rRNA;product=16S ribosomal RNA (parti...
5,coassemble_80719,barrnap:0.9,rRNA,41136,44022,0.000000e+00,+,.,Name=23S_rRNA;product=23S ribosomal RNA


In [13]:
# Write fasta file with 16S sequences
barrnap_16S = os.path.join(work_dir, "16S", 'barrnap_16S.fa')
!grep -A 1 "16S_rRNA" $barrnap_fasta > $barrnap_16S; cat $barrnap_16S

>16S_rRNA::coassemble_31453:0-878(-)
CTCCGGTTGATCCTGCCGGCGGCCACCGCTATAGGAATTCGATTAAGACATGCGAGTCGAGAGCGCAAGCTCGGCGGACTGCTCAGTAACACGTGGACAACGTGCCCTAAAGTGGGGGATAATCAGGGGAAACTCTGGATAATACCCCATAGATCATGAGATCTGGAATGACTTATGGTTCAAAGTTCCGGCGCTTTAGGATCGGTCTGCGGCCTATCAGGTAGTAGTGGGTGTAATGTACCCCCTAGCCTATTACGGGTATGGGCCTTGAGAGAGGGAGCCCAGAGTTGGATTCTGAGACACGAATCCAGGCCCTACGGGGCGCAGCAGTCGCGAAAACTTCACAATGGGCGAAAGCCCGATGAGGGAATTCCTAGTGCTAGCACTTTTGTGTTAGCTTTTCTTTAGCGTAGATAACTAGAGGAATAAGGGCTGGGTAAGACGGGTGCCAGCCGCCGCGGTAATACCCGCAGCCCGAGTGGTGGTCGATTTTATTGAGTCTAAAACGTTCGTAGCCGGTCTGATAGATCCTTGGGTAAATCGGGGGGCTTAACCTTCCGAATTCCGAGGAGACCGTCAGGCTTGGGATCGGGAGAGGTAAGAGGTACTTCAGGGGTAGGGGTAAAATCCTGTAATCCTTGGAGGACCACCGGTGGCGAAGGCGTCTTACTAGAACGAATCCGACGGTGAGGGACGAAGCCCTAGGTCGCAAACGGGATTAGATACCCCGGTAGTCTAGGGTGTAAACGCTGCAGACTTGGTGTTGGAGGCCCTTCGGGGGCATTCAGTGCCGGAGAGAAGTTGTTAAGTCTGCTACTTGGGGAGTACGTCCGCAAGGATGAAACTTAAAGGAATTGGCGGGGGAGCACCGCAACGGG
>16S_rRNA::coassemble_80719:1-680(+)
AGTGCCGGAGAGAAGTTGTTAAGTCTGCTACTTGGGGAGTACGTCCG

In [14]:
# Blast 16S sequences against SILVA 132 clustered at 97% identity
silva_result = os.path.join(work_dir, "16S", 'SILVA_blast.txt')
silva_cmd = 'blastn -query {query} -subject {subject} -outfmt 6 > {output}'
silva_cmd = silva_cmd.format(query = barrnap_16S, subject = silva_align, output = silva_result)
silva_job = 'bash -c "source activate {}; {}"'
silva_job = silva_job.format(anvio_env, silva_cmd)
print(silva_job)
!$silva_job

bash -c "source activate anvio; blastn -query /ebio/abt3_projects/vadinCA11/data/V11/16S/barrnap_16S.fa -subject /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/rep_set/rep_set_16S_only/97/silva_132_97_16S.fna -outfmt 6 > /ebio/abt3_projects/vadinCA11/data/V11/16S/SILVA_blast.txt"


In [15]:
# Generate tables with blast results
silva_df = pd.read_csv(silva_result, sep='\t', header=None)
silva_df.columns = ['qseqid','sseqid','pident','length','mismatch',
              'gapopen','qstart','qend','sstart','send','evalue','bitscore']


silva_seq1 = silva_df.loc[silva_df['qseqid'] == "16S_rRNA::coassemble_31453:0-878(-)"]
silva_seq2 = silva_df.loc[silva_df['qseqid'] == "16S_rRNA::coassemble_80719:1-680(+)"]

silva_seq1 = silva_seq1.sort_values(by=["pident"], ascending=False)
silva_seq2 = silva_seq2.sort_values(by=["pident"], ascending=False)

In [16]:
# Sequence 1
# Select matches with identity >= 95% and find their tax classification
seq1_tax = os.path.join(work_dir, "16S", '16S_seq1.tax')
silva_grep = 'bash -c "grep {0} {1}"'
for match in silva_seq1.loc[silva_df["pident"] >= 97][["sseqid"]].values:
    tax_id = '"{0}"'.format(match[0]) 
    grep_job = silva_grep.format(tax_id, silva_tax)
    print(grep_job)
    !$grep_job | sed 's/D_[0-9]__//g' >> $seq1_tax

bash -c "grep "AB237752.1.1428" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"
bash -c "grep "CBKQ010000059.83060.84525" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"


In [17]:
# Sequence 2
# Select matches with identity >= 97% and find their tax classification
seq2_tax = os.path.join(work_dir, "16S", '16S_seq2.tax')
silva_grep = 'bash -c "grep {0} {1}"'
for match in silva_seq2.loc[silva_df["pident"] >= 97][["sseqid"]].values:
    tax_id = '"{0}"'.format(match[0]) 
    grep_job = silva_grep.format(tax_id, silva_tax)
    print(grep_job)
    !$grep_job | sed 's/D_[0-9]__//g' >> $seq2_tax

bash -c "grep "CBKQ010000059.83060.84525" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"
bash -c "grep "KJ522703.1.1246" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"
bash -c "grep "JF807263.1.1257" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"
bash -c "grep "KU754011.1.1256" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"
bash -c "grep "JF807280.1.1256" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"
bash -c "grep "JF807141.1.1256" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majority_taxonomy_7_levels.txt"
bash -c "grep "JF807311.1.1257" /ebio/abt3_projects/databases/SILVA/SILVA_132_QIIME_release/taxonomy/16S_only/97/majorit

In [18]:
# Print the 7 level taxonomic classification of the top hits
seq_1_df = pd.read_csv(seq1_tax, sep='\t|;', skiprows=0, header=None)
seq_1_df.columns = ['seqname','Kingdom','Phylum','Class','Order', 'Family', 
                       'Genus','Species']
seq_1_df

/ebio/abt3_projects/software/miniconda3/envs/qiime2/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,seqname,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,AB237752.1.1428,Archaea,Euryarchaeota,Thermoplasmata,uncultured,uncultured archaeon,uncultured archaeon,uncultured archaeon
1,CBKQ010000059.83060.84525,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,Ambiguous_taxa
2,KJ522703.1.1246,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,Candidatus Methanomethylophilus alvus
3,KU754009.1.1257,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
4,JF807078.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,Ambiguous_taxa
5,JF807279.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
6,JF807141.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
7,KU754014.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
8,CP014214.1550807.1552273,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,Ambiguous_taxa
9,JF807281.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon


In [19]:
# Print the 7 level taxonomic classification of the top hits
seq_2_df = pd.read_csv(seq2_tax, sep='\t|;', skiprows=0, header=None)
seq_2_df.columns = ['seqname','Kingdom','Phylum','Class','Order', 'Family', 
                       'Genus','Species']
seq_2_df

/ebio/abt3_projects/software/miniconda3/envs/qiime2/lib/python3.5/site-packages/ipykernel/__main__.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  from ipykernel import kernelapp as app


,seqname,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,CBKQ010000059.83060.84525,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,Ambiguous_taxa
1,KJ522703.1.1246,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,Candidatus Methanomethylophilus alvus
2,JF807263.1.1257,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
3,KU754011.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
4,JF807280.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
5,JF807141.1.1256,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
6,JF807311.1.1257,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
7,KM213864.1.1403,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
8,JF807283.1.1257,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon
9,JF807167.1.1259,Archaea,Euryarchaeota,Thermoplasmata,Methanomassiliicoccales,Methanomethylophilaceae,Candidatus Methanomethylophilus,uncultured archaeon


limit_output extension: Maximum message size of 10000 exceeded with 11790 characters

# Assessment of bin assembly
* Aim: to assess the quality of the assembly of the **vadinCA11** genome bin.

For this I will use both CheckM and Quast.

## CheckM

### Running CheckM

In [61]:
# Running CheckM to verify the completness and redundancy of the V11 bin
#
# CheckM requires a folder with the bins and doesn't allow a single file as input
# Therefore, I provided the folder with all the bins as input, but especified 
# that the extension of the file is "478.contigs.fa", limiting the analysis to two files
# One of which is the vadinCA11 bin I'm interested in
#
checkm_file = os.path.join(work_dir, "checkm_output", 'CheckM_V11.txt')
checkm_cmd = """checkm lineage_wf -t {0} \
    -x {1} \
    -f {2} \
    {3} {4}/checkm_output """
checkm_cmd = checkm_cmd.format(8, "478.contigs.fa", checkm_file, V11_folder, work_dir)
checkm_job = 'bash -c "source activate {0}; {1}"'
checkm_job = checkm_job.format(quality_env, checkm_cmd)
print(checkm_job)
!$checkm_job

bash -c "source activate py2_genome_quality; checkm lineage_wf -t 8     -x 478.contigs.fa     -f /ebio/abt3_projects/vadinCA11/data/V11/checkm_output/CheckM_V11.txt     /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/ /ebio/abt3_projects/vadinCA11/data/V11/checkm_output "

*******************************************************************************
 [CheckM - tree] Placing bins in reference genome tree.
*******************************************************************************

  Identifying marker genes in 2 bins with 8 threads:
    Finished processing 2 of 2 (100.00%) bins.
  Saving HMM info to file.

  Calculating genome statistics for 2 bins with 8 threads:
    Finished processing 2 of 2 (100.00%) bins.

  Extracting marker genes to align.
  Parsing HMM hits to marker genes:
    Finished parsing hits for 2 of 2 (100.00%) bins.
  Extracting 43 HMMs with 8 threads:
    Finished extracting 43 of 43 (100.00%) HMMs.
  Alig

### CheckM results

In [62]:
checkm_stats = "{0}/checkm_output/storage/bin_stats_ext.tsv".format(work_dir)
checkm_df = pd.read_csv(checkm_stats, sep='\t', skiprows=0, header=None)
checkm_df = eval(checkm_df.iat[0,1]) # Select only the stats from the V11 bin and turn into dict
checkm_df = pd.DataFrame(list(checkm_df.items()))
checkm_df.columns = ["Stat", "Value"]
checkm_df = checkm_df.set_index('Stat')
checkm_df.loc[["Completeness", "Contamination", "Genome size", "# ambiguous bases", 
               "# contigs", "Mean contig length", "N50 (contigs)", "Longest contig",
               "# scaffolds", "Mean scaffold length", "N50 (scaffolds)",
               "Longest scaffold", "GC", "GC std", "# predicted genes", 
               "Translation table", "Coding density", "marker lineage", "# genomes",
               "# markers", "# marker sets" ]]

,Value
Stat,
Completeness,98.3871
Contamination,0
Genome size,1590932
# ambiguous bases,0
# contigs,48
Mean contig length,33144.4
N50 (contigs),50438
Longest contig,103078
# scaffolds,48


## Quast

### Running Quast

In [70]:
# Running Quast on the bin
quast_cmd = """quast {0} \
    -o {1}/quast_output \
    --gene-finding \
    --glimmer \
    --threads {2}"""
quast_cmd = quast_cmd.format(V11_bin, work_dir, 8)
quast_job = 'bash -c "source activate {0}; {1}"'
quast_job = quast_job.format(quality_env, quast_cmd)
print(quast_job)
!$quast_job

bash -c "source activate py2_genome_quality; quast /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/metabat2_low_PE.478.contigs.fa     -o /ebio/abt3_projects/vadinCA11/data/V11/quast_output     --gene-finding     --glimmer     --threads 8"
/ebio/abt3_projects/software/miniconda3/envs/py2_genome_quality/opt/quast-4.6.3/quast.py /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/metabat2_low_PE.478.contigs.fa -o /ebio/abt3_projects/vadinCA11/data/V11/quast_output --gene-finding --glimmer --threads 8

Version: 4.6.3

System information:
  OS: Linux-4.4.67-x86_64-with-debian-stretch-sid (linux_64)
  Python version: 2.7.14
  CPUs number: 80

Started: 2018-06-06 12:42:26

Logging to /ebio/abt3_projects/vadinCA11/data/V11/quast_output/quast.log
NOTICE: Output directory already exists. Existing Nucmer alignments can be used

CWD: /ebio/abt3_projects/vadinCA11/notebooks/metagenome/assem

### Quast results

In [72]:
quast_stats = "{0}/quast_output/report.tsv".format(work_dir)
quast_df = pd.read_csv(quast_stats, sep='\t', skiprows=0)
quast_df = quast_df.set_index('Assembly')
quast_df

,metabat2_low_PE.478.contigs
Assembly,
# contigs (>= 0 bp),48.00
# contigs (>= 1000 bp),48.00
# contigs (>= 5000 bp),47.00
# contigs (>= 10000 bp),40.00
# contigs (>= 25000 bp),25.00
# contigs (>= 50000 bp),14.00
Total length (>= 0 bp),1590932.00
Total length (>= 1000 bp),1590932.00
Total length (>= 5000 bp),1587803.00


# tRNA gene content

In [78]:
# Running aragorn on the bin
aragorn_file = os.path.join(work_dir, "tRNA_output", 'V11_tRNAs.fasta')
aragorn_cmd = """aragorn -gc11 -fons -o {0} {1}"""
aragorn_cmd = aragorn_cmd.format(aragorn_file, V11_bin)
aragorn_job = 'bash -c "source activate {0}; {1}"'
aragorn_job = aragorn_job.format(metacompass_env, aragorn_cmd)
print(aragorn_job)
!$aragorn_job

bash -c "source activate metacompass; aragorn -gc11 -fons -o /ebio/abt3_projects/vadinCA11/data/V11/tRNA_output/V11_tRNAs.fasta /ebio/abt3_projects/vadinCA11/data/metagenome/LLMGA/HiSeqRun83-91/bin_refine/DAS_Tool//bins_DASTool_bins/metabat2_low_PE.478.contigs.fa"


In [79]:
# Count and print names of the tRNA genes detected
!grep -c ">" $aragorn_file
!grep ">" $aragorn_file

40
>1-1tRNA-Thr(tgt)c[72,148]
>1-2tRNA-Arg(cct)[2895,2972]
>2-1tRNA-Leu(tag)[44244,44329]
>3-1tRNA-Ser(tga)[46845,46930]
>4-1tRNA-Ser(gga)c[15630,15716]
>5-1tRNA-Ser(gct)[1523,1608]
>5-2tRNA-Leu(taa)c[64902,64987]
>6-1tRNA-Pyl(cta)c[29512,29582]
>7-1tRNA-Gly(tcc)[33795,33870]
>10-1tRNA-Asp(gtc)[29110,29185]
>11-1tRNA-Met(cat)c[12359,12436]
>11-2tRNA-Gln(ctg)c[50830,50905]
>11-3tRNA-Ala(ggc)[72006,72079]
>11-4tRNA-Glu(ctc)[102867,102940]
>12-1tRNA-Pro(ggg)c[7914,7989]
>14-1tRNA-Pro(tgg)c[3136,3209]
>16-1tRNA-Thr(tgt)c[55396,6]
>20-1tRNA-Arg(gcg)[16426,16502]
>21-1tRNA-Gln(ttg)c[1842,1917]
>21-2tRNA-Ala(tgc)[14706,14779]
>21-3tRNA-Leu(caa)[46349,46434]
>21-4tRNA-His(gtg)c[84599,84673]
>22-1tRNA-Arg(ccg)c[8353,8427]
>24-1tRNA-Gly(gcc)[3797,3871]
>27-1tRNA-Leu(cag)[384,470]
>27-2tRNA-Glu(ttc)[18094,18169]
>28-1tRNA-Val(gac)[7911,7986]
>28-2tRNA-Ser(cga)[11120,11205]
>30-1tRNA-Thr(cgt)[31738,31812]
>30-2tRNA-Phe(gaa)[38091,38166]
>30-3tRNA-Met(cat)c[45494,45570]
>30-4tRNA-Asn(gtt)c[45673,45

Of the 20 standard aminoacids, 17 were found; Isoleucine (Ile), Tyrosine (Tyr) and Tryptophan (Trp) were not detected. In addition, a tRNA for the non-standar amino acid pyrrolysine  was also detected	

# sessionInfo

In [1]:
!conda list -n py2_genome_quality

# packages in environment at /ebio/abt3_projects/software/miniconda3/envs/py2_genome_quality:
#
backports                 1.0                      py27_1    conda-forge
backports.functools_lru_cache 1.5                      py27_0    conda-forge
backports_abc             0.5                      py27_0    conda-forge
bcftools                  1.6                           1    bioconda
blast                     2.7.1               boost1.64_3    bioconda
boost                     1.64.0                   py27_4    conda-forge
boost-cpp                 1.64.0                        1    conda-forge
bz2file                   0.98                     py27_0  
bzip2                     1.0.6                         1    conda-forge
ca-certificates           2018.1.18                     0    conda-forge
centrifuge                1.0.3            py27pl5.22.0_2    bioconda
certifi                   2018.1.18                py27_0    conda-forge
checkm-genome             1.0.7               

In [2]:
!conda list -n anvio

# packages in environment at /ebio/abt3_projects/software/miniconda3/envs/anvio:
#
anvio                     4.0.0                    py35_1    bioconda
asn1crypto                0.22.0                   py35_0    conda-forge
bcftools                  1.4.1                         0    bioconda
blast                     2.2.31                        1    bioconda
boost                     1.65.1                   py35_0    conda-forge
boost-cpp                 1.65.1                        1    conda-forge
bottle                    0.12.13                  py35_0    conda-forge
bwa                       0.7.15                        1    bioconda
bzip2                     1.0.6                         1    conda-forge
ca-certificates           2017.11.5                     0    conda-forge
cairo                     1.14.6                        5    conda-forge
centrifuge                1.0.3            py35pl5.22.0_1    bioconda
certifi                   2017.11.5                py35_

In [3]:
!conda list -n metacompass

# packages in environment at /ebio/abt3_projects/software/miniconda3/envs/metacompass:
#
anvio                     4.0.0                    py35_2    bioconda
aragorn                   1.2.38                        1    bioconda
asn1crypto                0.24.0                   py35_0  
atomicwrites              1.1.5                    py35_0  
attrs                     18.1.0                   py35_0  
backcall                  0.1.0                    py35_0  
barrnap                   0.9                           0    bioconda
bcftools                  1.4.1                         0    bioconda
bcrypt                    3.1.4            py35ha35c455_0  
bedtools                  2.27.1                        1    bioconda
biom-format               2.1.6                    py35_1    bioconda
biopython                 1.68                     py35_0    bioconda
blast                     2.7.1                h96bfa4b_5    bioconda
blast-legacy              2.2.26                   